In [1]:
import pandas as pd
from statsmodels.multivariate.pca import PCA
import dcor
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Legend
from bokeh.palettes import Spectral6, Dark2, inferno
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Range1d

import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

output_notebook()

Loading BokehJS ...

# Load data

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1oBcTHFHMJqFgzIMdROsdDJ3B8o2-FUDOIdJjMnhxPZY'
RANGE = 'wodlog!A:AG'

# Get credentials
creds = None
if os.path.exists('token.pickle'):
    with open('/Users/hasannagib/Documents/gsheet_api/token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '/Users/hasannagib/Documents/gsheet_api/autowod_creds.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('/Users/hasannagib/Documents/gsheet_api/token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Get data
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(
    spreadsheetId=SPREADSHEET_ID,
    range=RANGE
).execute()
values = result.get('values', [])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=130515744428-6in65u9i9vhickestc850on3peqmlc7u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53802%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=d56Qk10LmB22Xbic4F494JpGdSF5pT&access_type=offline


In [3]:
df = pd.DataFrame(values[1:], columns=values[0]).fillna(0).replace('',0)
df = df.astype({
    'day':int,
    'ex':object,
    'wt':float, 
    'rep':float,
    'group':float,
    'duration':float,
    'dist(m)':float,
    'cals':float,
    'emom':float,
    'tabata':int,
    'work':float,
    'rest':float,
    'deathby':float,
    'master':float,
    'time':float,
    'task':float,
    'gymnastics':float,
    'mmetcon':float,
    'wtlft':float,
    'bilateral':float, 
    'unilateral':float, 
    'push':float,
    'pull':float, 
    'chest':float, 
    'back':float, 
    'shoulder':float, 
    'bi':float, 
    'tri':float, 
    'quad':float, 
    'ham':float, 
    'hip':float,
    'core':float, 
}
)
df['vol'] = df['wt']*df['rep']
df = df.set_index(['ex', 'day', 'dt', 'wt', 'rep'])
df = df.multiply(df['vol'], axis='index').drop(columns=['group', 'vol'])

In [4]:
# modality & muscle groups only
df = df.iloc[:, 8:]

# modality and workout styles only
#df = df.iloc[:, :11]

df = df.groupby(['day', 'ex']).sum()
df.columns

Index(['master', 'time', 'task', 'gymnastics', 'mmetcon', 'wtlft', 'bilateral',
       'unilateral', 'push', 'pull', 'chest', 'back', 'shoulder', 'bi', 'tri',
       'quad', 'ham', 'hip', 'core'],
      dtype='object')

# Compare pairwise WOD `dCorr` values

In [ ]:
cors = []
for i in range(df.index.max()[0]+1):
    for j in range(df.index.max()[0]+1):
        cor = dcor.distance_correlation(
            df.query(f'day=={i}').T, 
            df.query(f'day=={j}').T
        )
    
        cors.append([i, j, cor])
        
df_corr = pd.DataFrame(cors, columns=['day_A', 'day_B', 'corr'])
df_corr = df_corr.set_index(['day_A', 'day_B']).reset_index()

## Visualizing WOD Similarity

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_corr.pivot('day_A', 'day_B', 'corr'), cmap="Reds", annot=True);

### Interpreting the heatmap

The matrix above shows similarity among pairwise WODs. 1 being identical and 0 being least similar (or rest days). e.g. If we consider the bottom row, and read it from right to left, we see that day 12 has very little in common with day 8,9,10. But it is pretty close to day 5 & 6. Let's see if this makes intuitive sense or not by examining the WODs from those days...

In [ ]:
df.query('day==14')

In [ ]:
df.query('day==6')

In [ ]:
df.query('day==2')

## Comparing multiple WODs

We can also compare workout from one day to multiple other days. Or one week to another week.

In [ ]:
# comparing day 12 against days 0,1,2,4,5,6
dcor.distance_correlation(
    df.query(f'day in (0,1,2,4,5,6)').T, 
    df.query(f'day in (12,)').T
)

In [ ]:
dcor.distance_correlation(
    df.query(f'day in (0,1,2)').T, 
    df.query(f'day in (4,5,6)').T
)

In [ ]:
dcor.distance_correlation(
    df.query(f'day in (0,1,2)').T, 
    df.query(f'day in (8,9,10)').T
)

In [ ]:
dcor.distance_correlation(
    df.query(f'day in (4,5,6)').T, 
    df.query(f'day in (8,9,10)').T
)

In [ ]:
dcor.distance_correlation(
    df.query(f'day in (12,13,14)').T, 
    df.query(f'day in (8,9,10)').T
)